# 要約 
このJupyterノートブックでは、4ビット量子化された「Gemma-2 9b Instruct」モデルに基づくLoRAアダプターを使用した推論の手法が示されています。主な目的は、Chatbot Arenaコンペティションにおける、2つの言語モデル間のユーザー応答の選好予測を行うことです。

### 問題の扱い
ノートブックは、LoRAアダプターを用いて、量子化による誤差の影響を軽減しつつ、推論を迅速化する方法を探求しています。モデルをマージすることで誤差が生じる可能性があるため、LoRAアダプターを維持した状態での推論を推奨しています。また、モデルの性能は評価セットでの対数損失が0.9371、公開リーダーボードでの対数損失が0.941であると報告しています。

### 使用ライブラリと手法
ノートブックでは、主に以下のライブラリが使用されています:
- **Transformers**: 「Gemma2ForSequenceClassification」と「GemmaTokenizerFast」を使用してモデルの呼び出しやトークン化を実施。
- **Peft**: LoRAアダプターの適用に用いられます。
- **Torch**: GPU利用や自動混合精度計算をサポートし、モデルの推論が行われます。
- **PandasとNumPy**: データ処理と操作のためにデータフレームを使用し、結果の格納に役立てています。

### データ処理と推論
データはCSVファイルから読み込まれ、テキストの前処理を行った後、トークナイズされます。推論は、2つのGPUを用いてバッチごとに行われ、モデルAとモデルBの各々の勝率を計算します。結果として、モデルごとの勝率や引き分け確率をデータフレームに格納し、最終的に提出用のCSVファイルとして成果物を保存します。

全体として、このノートブックは、量子化されたモデルを最適に利用するための手法と細かな設定を提供し、効率的に予測を行うフレームワークを構築しています。

---


# 用語概説 
以下は、Jupyter Notebook内で使用されている専門用語の簡単な解説です。これらの用語は、初心者が特に理解しにくい可能性があるものです。

1. **Gemma-2**: 大規模言語モデル（LLM）の一種で、テキスト生成や分類タスクに用いられます。特に今回のノートブックでは、「Gemma-2 9b Instruct」というバージョンが使用されています。

2. **LoRA (Low-Rank Adaptation)**: 転移学習やファインチューニングを行うための手法で、もともとのモデルの重みを劣化させることなく、新しい情報を効率的に学習するために低ランクの更新を加える手法です。

3. **量子化 (Quantization)**: モデルの重みやアクティベーションの精度を減らして、メモリ使用量を削減し、計算効率を向上させる技術です。特に4ビット量子化は、重みを4ビットで表現することを意味します。

4. **アテンションマスク (Attention Mask)**: モデルに入力されるデータのどの部分を注目すべきかを示すバイナリのマスクです。通常は、パディングされた部分（無視すべき部分）を除外するために使われます。

5. **トークナイズ (Tokenization)**: 文章やテキストを処理可能な単位（トークン）に分解するプロセスです。これにより、テキストがモデルに入力できる形式に変換されます。

6. **トークンID (Token ID)**: トークナイズされた各トークンに対応する整数値で、単語やサブワードを表現します。

7. **ソフトマックス (Softmax)**: 複数のクラスに対する確率を計算する関数です。出力層で使用され、各クラスのスコアを確率に変換します。

8. **TTA (Test-Time Augmentation)**: テストデータに対してデータ拡張を適用する手法で、モデルの予測の安定性を向上させます。複数の異なる視点やデータの拡張から得られる予測を平均化することが一般的です。

9. **Pad_without_fast_tokenizer_warning**: 特定のトークナイザー使用時に、パディングに関する警告を避けるための関数です。入力のパディングを行う際には、慎重に扱う必要があります。

10. **自動混合精度 (Automatic Mixed Precision)**: 計算の精度を自動的に調整する技術で、処理速度を向上させるために、必要に応じて32ビット（float32）と16ビット（float16）の演算を切り替えます。

11. **キャッシュ (Cache)**: 計算結果を保存しておき、再使用することで計算を効率化すること。特に推論時間を短縮するために有効です。

これらの用語は、特にこのノートブックやその設定において重要な部分を占めており、理解を深める手助けとなります。

---


## このノートブックについて

これは、4ビット量子化された [Gemma-2 9b Instruct](https://blog.google/technology/developers/google-gemma-2/) と、私がアップロードしたスクリプトを使用してトレーニングしたLoRAアダプターを利用した推論ノートブックです [ここ](https://www.kaggle.com/code/emiz6413/gemma-2-9b-4-bit-qlora-finetune)で確認できます。
LoRAアダプターをベースモデルにマージすることで推論を速くすることもできますが、安易にそうすると無視できない量子化誤差が発生する可能性があります。そのため、私はLoRAアダプターをマージせずに維持することにしました。

## 結果

| サブセット | 対数損失 |
| - | - |
| 評価セット | 0.9371 |
| 公開LB | 0.941 |

提出には約4時間かかります。`max_length=2048`でTTAは使用していません。


In [ ]:
!pip install transformers peft accelerate bitsandbytes \
    -U --no-index --find-links /kaggle/input/lmsys-wheel-files

In [ ]:
import time
from dataclasses import dataclass
from concurrent.futures import ThreadPoolExecutor

import torch
import sklearn
import numpy as np
import pandas as pd
from transformers import Gemma2ForSequenceClassification, GemmaTokenizerFast, BitsAndBytesConfig
from transformers.data.data_collator import pad_without_fast_tokenizer_warning
from peft import PeftModel

In [ ]:
assert torch.cuda.device_count() == 2

## 設定


In [ ]:
@dataclass
class Config:
    gemma_dir = '/kaggle/input/gemma-2/transformers/gemma-2-9b-it-4bit/1/gemma-2-9b-it-4bit'
    lora_dir = '/kaggle/input/73zap2gx/checkpoint-5748'
    max_length = 2048
    batch_size = 4
    device = torch.device("cuda")    
    tta = False  # テスト時のデータ拡張。<prompt>-<model-bの応答>-<model-aの応答>
    spread_max_length = False  # 各入力にmax_length//3を適用するか、連結した入力にmax_lengthを適用するか

cfg = Config()

# データの読み込みと前処理


In [ ]:
test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')

In [ ]:
def process_text(text: str) -> str:
    return " ".join(eval(text, {"null": ""}))

test.loc[:, 'prompt'] = test['prompt'].apply(process_text)
test.loc[:, 'response_a'] = test['response_a'].apply(process_text)
test.loc[:, 'response_b'] = test['response_b'].apply(process_text)

display(test.head(5))

# トークナイズ


In [ ]:
def tokenize(
    tokenizer, prompt, response_a, response_b, max_length=cfg.max_length, spread_max_length=cfg.spread_max_length
):
    prompt = ["<prompt>: " + p for p in prompt]  # プロンプトに"<prompt>: "を追加
    response_a = ["\n\n<response_a>: " + r_a for r_a in response_a]  # 応答Aにプレフィックスを追加
    response_b = ["\n\n<response_b>: " + r_b for r_b in response_b]  # 応答Bにプレフィックスを追加
    if spread_max_length:  # spread_max_lengthがTrueの場合
        # 各要素をmax_length//3でトークナイズ
        prompt = tokenizer(prompt, max_length=max_length//3, truncation=True, padding=False).input_ids
        response_a = tokenizer(response_a, max_length=max_length//3, truncation=True, padding=False).input_ids
        response_b = tokenizer(response_b, max_length=max_length//3, truncation=True, padding=False).input_ids
        input_ids = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]  # 各リストを結合
        attention_mask = [[1]* len(i) for i in input_ids]  # 各入力の長さに応じたアテンションマスクを作成
    else:
        # 各要素を結合してトークン化
        text = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]
        tokenized = tokenizer(text, max_length=max_length, truncation=True, padding=False)  # トークナイズ
        input_ids = tokenized.input_ids  # トークンIDを取得
        attention_mask = tokenized.attention_mask  # アテンションマスクを取得
    return input_ids, attention_mask  # トークンIDとアテンションマスクを返す

In [ ]:
%%time

tokenizer = GemmaTokenizerFast.from_pretrained(cfg.gemma_dir)  # Gemmaトークナイザーを読み込む
tokenizer.add_eos_token = True  # 終了トークンを追加
tokenizer.padding_side = "right"  # パディングの位置を右に設定

data = pd.DataFrame()
data["id"] = test["id"]
data["input_ids"], data["attention_mask"] = tokenize(tokenizer, test["prompt"], test["response_a"], test["response_b"])  # トークン化した結果をデータフレームに格納
data["length"] = data["input_ids"].apply(len)  # 各入力の長さを計算して追加

aug_data = pd.DataFrame()  # 拡張データ用のデータフレームを作成
aug_data["id"] = test["id"]
# response_aとresponse_bを入れ替える
aug_data['input_ids'], aug_data['attention_mask'] = tokenize(tokenizer, test["prompt"], test["response_b"], test["response_a"])  # トークナイズして格納
aug_data["length"] = aug_data["input_ids"].apply(len)  # 長さを計算

In [ ]:
print(tokenizer.decode(data["input_ids"][0]))  # トークナイズしたデータの最初の要素をデコードして表示

In [ ]:
print(tokenizer.decode(aug_data["input_ids"][0]))  # 拡張データの最初の要素をデコードして表示

# モデルを読み込む


In [ ]:
# GPU 0にベースモデルを読み込む
device_0 = torch.device('cuda:0')
model_0 = Gemma2ForSequenceClassification.from_pretrained(
    cfg.gemma_dir,
    device_map=device_0,
    use_cache=False,
)

# GPU 1にベースモデルを読み込む
device_1 = torch.device('cuda:1')
model_1 = Gemma2ForSequenceClassification.from_pretrained(
    cfg.gemma_dir,
    device_map=device_1,
    use_cache=False,
)

#### LoRAアダプターを読み込む


In [ ]:
model_0 = PeftModel.from_pretrained(model_0, cfg.lora_dir)  # LoRAアダプターをモデル0に適用
model_1 = PeftModel.from_pretrained(model_1, cfg.lora_dir)  # LoRAアダプターをモデル1に適用

# 推論



In [ ]:
@torch.no_grad()  # 勾配計算を無効にする
@torch.cuda.amp.autocast()  # 自動混合精度を使って演算を行う
def inference(df, model, device, batch_size=cfg.batch_size, max_length=cfg.max_length):
    a_win, b_win, tie = [], [], []  # 各モデルの勝率と引き分けを記録するリスト
    
    for start_idx in range(0, len(df), batch_size):
        end_idx = min(start_idx + batch_size, len(df))  # バッチの終端インデックスを計算
        tmp = df.iloc[start_idx:end_idx]  # データフレームからバッチを取得
        input_ids = tmp["input_ids"].to_list()  # 入力IDを取得
        attention_mask = tmp["attention_mask"].to_list()  # アテンションマスクを取得
        # トークナイザーを使ってデータをパディング
        inputs = pad_without_fast_tokenizer_warning(
            tokenizer,
            {"input_ids": input_ids, "attention_mask": attention_mask},
            padding="longest",
            pad_to_multiple_of=None,
            return_tensors="pt",
        )
        outputs = model(**inputs.to(device))  # モデルで出力を計算
        proba = outputs.logits.softmax(-1).cpu()  # ロジットをソフトマックスで確率に変換
        
        a_win.extend(proba[:, 0].tolist())  # モデルAの勝率をリストに追加
        b_win.extend(proba[:, 1].tolist())  # モデルBの勝率をリストに追加
        tie.extend(proba[:, 2].tolist())  # 引き分けの確率をリストに追加
    
    df["winner_model_a"] = a_win  # モデルAの勝率をデータフレームに追加
    df["winner_model_b"] = b_win  # モデルBの勝率をデータフレームに追加
    df["winner_tie"] = tie  # 引き分けの確率をデータフレームに追加
    
    return df  # 更新されたデータフレームを返す

In [ ]:
st = time.time()  # 処理開始時間を記録

# 入力の長さでソートして動的パディングを最大限に活用する
data = data.sort_values("length", ascending=False)
# sub_1とsub_2のトークン数がほぼ同じになるように分ける
sub_1 = data.iloc[0::2].copy()  # 偶数番目のデータを選択
sub_2 = data.iloc[1::2].copy()  # 奇数番目のデータを選択

with ThreadPoolExecutor(max_workers=2) as executor:
    results = executor.map(inference, (sub_1, sub_2), (model_0, model_1), (device_0, device_1))  # 2つのモデルで推論実行

result_df = pd.concat(list(results), axis=0)  # 結果を結合
proba = result_df[["winner_model_a", "winner_model_b", "winner_tie"]].values  # 勝率の配列を取得

print(f"経過時間: {time.time() - st}")  # 処理時間を表示

In [ ]:
st = time.time()

if cfg.tta:  # TTAが有効な場合
    data = aug_data.sort_values("length", ascending=False)  # 入力の長さでソートしてスピードを向上させる
    sub_1 = data.iloc[0::2].copy()  # 偶数番目のデータ
    sub_2 = data.iloc[1::2].copy()  # 奇数番目のデータ

    with ThreadPoolExecutor(max_workers=2) as executor:
        results = executor.map(inference, (sub_1, sub_2), (model_0, model_1), (device_0, device_1))  # 2つのモデルで推論実行

    tta_result_df = pd.concat(list(results), axis=0)  # TTAの結果を結合
    # TTAの順序が反転するので調整
    tta_proba = tta_result_df[["winner_model_b", "winner_model_a", "winner_tie"]].values 
    # 元の結果とTTA結果を平均
    proba = (proba + tta_proba) / 2

print(f"経過時間: {time.time() - st}")  # 処理時間を表示

In [ ]:
result_df.loc[:, "winner_model_a"] = proba[:, 0]  # モデルAの勝率を更新
result_df.loc[:, "winner_model_b"] = proba[:, 1]  # モデルBの勝率を更新
result_df.loc[:, "winner_tie"] = proba[:, 2]  # 引き分けの勝率を更新
submission_df = result_df[["id", 'winner_model_a', 'winner_model_b', 'winner_tie']]  # 提出用のデータフレームを作成
submission_df.to_csv('submission.csv', index=False)  # CSVファイルに保存
display(submission_df)  # 提出データを表示

---

# コメント 

> ## Cody_Null
> 
> 推論時間を速めるアイデアはありますか？パフォーマンスを失わずに。
> 
> 
> > ## Eisuke Mizutaniトピック作成者
> > 
> > LoRA_A x LoRA_Bを最初に計算したときにキャッシュするのは簡単な方法ですが、それほど速度向上は見込めないかもしれません。
> > 
> > TensorRTやvLLMのような最適化ライブラリを使えるのかも気になります。
> > 
> > 
> > > ## Cody_Null
> > > 
> > > vLLMを試したことがありますか？私は試しましたが、うまく動かす方法がわかりませんでした。 
> > > 
> > > 
> > > 
> > > ## Eisuke Mizutaniトピック作成者
> > > 
> > > まだ試していません。max_lengthを増やすと対数損失が減少することは認識していますが、2048を越えると改善は非常に小さいです。私のケースでは2048から4096にすると対数損失が0.002減少しました。残りの時間で最適化できる他の方法を検討しています。
> > > 
> > > 

---

> ## carvingfate
> 
> 私は以前30位でしたが、このコードのおかげで私の努力が無駄に思えてしまいます。しかし、共有する精神を尊重しており、これがインターネットの精神であると思います。
> 
> 
> > ## jointcc2
> > 
> > 業界の状況もそうだと思います、一つのモデルが全ての過去の努力を上回りますね。
> > 
> > 

---

> ## Van chrn
> 
> なぜvLLMではなく？それはもっと速いかもしれません！
> 
> 
> > ## Eisuke Mizutaniトピック作成者
> > 
> > 私はそれに取り組んでいます！
> > 
> > 
> > > ## Cody_Null
> > > 
> > > この使用方法を実現しましたか？私はvLLMを使ったことがなく、動作しているのを見てみたいです！
> > > 
> > > 

---

> ## Dai LinLing
> 
> 共有してくれてありがとう。これは私にとって非常に助けになり、理解も深まりました。
> 
> 

---

> ## Turbo
> 
> [@emiz6413](https://www.kaggle.com/emiz6413)   ノートブックを共有してくれてありがとう。
> 
> 

---

> ## Vitalii Bozheniuk
> 
> なぜシルバーティアの解決策を公開するのかわかりません。この0.88のノートブックを公開すれば、全員が1位になれるのですか？人々がアイデアやノートブックを共有するのは理解できますが、30位のノートブックを共有するのは意味がありません。競争とチャレンジの雰囲気が消えてしまいます。
> 
> 
> > ## G John Rao
> > 
> > まだ1ヶ月残っていますが、初心者にとってはブーストになります。経験豊富な専門家にとっては、1ヶ月は新しいアイデアを構築したり実装するには十分な時間です。 
> > 
> > 

---

> ## Korey Ma
> 
> [@emiz6413](https://www.kaggle.com/emiz6413) ノートブックをありがとう！私はいくつかの追加パラメータを微調整し、cv&lb(0.912&0.924)を達成しました。さらに良くするために他のトリックを試したいです😆
> 
> 
> > ## Yichuan Gao
> > 
> > もう少し詳細を共有していただけますか？ハイパーパラメータを調整しているのか、LoRAレイヤーやランクを増やしているのか？
> > 
> > > ## Korey Ma
> > > 
> > > "o_proj"と"gate_proj"を追加しただけです。 
> > > 

---

> ## samson
> 
> かなり良いノートブックを作成されていて、コメントに対する見解も妥当です。
> 
> ですが、なぜ重みを共有したのでしょう？真剣に学びたいと思っている人は、あなたの方法を使ったりそれを自分に適応したりするでしょう。しかし、あなたのせいで100以上のチームが盲目的にコピー+提出することになりました。これにより、中間にいる人々が適切なチームメートを見つけるのは不可能です。
> 
> 

---

> ## superferg
> 
> すごいですね。
> 
> 

---

> ## yuanzhe zhou
> 
> よくやりました！つまり、LLMを使うことが鍵ですか？BERTタイプのモデルは収束するには小さすぎるようです。
> 
> > ## Valentin Werner
> > 
> > まさにそうです。私もLLMがシーケンス生成で十分に微調整されていると思います。AI生成テキストをより認識し、テキストがどうあるべきかに最適化されているため、このタスクに適しています。これにより、基本的に火に火をもって戦うことになります。
> > 
> > > ## Eisuke Mizutaniトピック作成者
> > > 
> > > 実際、deberta-v3-smallを完全に微調整したところ、約1.1になりました。
> > > > BERTスタイルのエンコーダアーキテクチャは、理論的にはこれらの分類タスクにより適していると思います。
> > > しかし、あなたが指摘したように、実際にはLLMははるかに大きい（deberta v2 xxlargeは1.5B）ため、過学習を避けることができ、より多くのメモリ容量を持つことができます。
> > > もう一つの理由は、指示微調整という、非常に競技に似たデータを使用しているからかもしれません。
> > > 私はバニラgemma-2-9bでテストしたことはありませんが、どのように動作するかを見るのは興味深いです。
> > > 

---

> ## ano
> 
> [@emiz6413](https://www.kaggle.com/emiz6413) ノートブックをありがとう！微調整したモデルの検証データとcvスコアについて教えていただけますか？あなたのトレーニングノートブックに基づいて、私は行数を5で割った数に基づいて、約20％のデータを検証用に使用しました。その後、対数損失を計算しましたが、cvスコアは0.9未満でした。明らかに、検証データに間違いがあったため、cvスコアはあなたのトレーニングノートブックで書かれた0.9371よりも低くなりました。微調整モデルの検証データをどのように作成すればよいか教えていただけますか？
> 
> > ## Eisuke Mizutaniトピック作成者
> > 
> > 私のトレーニングノートブックで検証データを作成するには、次の行が実行されないことを確認してください。
> > 
> > ```
> > # ds = ds.select(torch.arange(100))
> > 
> > ```
> > 
> > 次に、最後のセルにあるこの行で検証セットを選択する必要があります。
> > 
> > ```
> > ds.select(eval_idx)
> > 
> > ```
> > 
> > > ## ano
> > > 
> > > 返信ありがとうございます。もちろん、データを減らすための行を削除しました。
> > > 
> > > では、検証データはあなたのトレーニングノートブックでのn_splits = 5およびfold_idx = 0で選択されるのですね。うーん、そうすると、私のコードにCVスコアを計算する上での間違いがあったかもしれません。 
> > > > [UPDATE] バグを見つけました。ありがとうございます！
> > > 

---

> ## Guillermo Perez G
> 
> 素晴らしい！しかし、スコアをさらに下げることはできないと思います。それとも、ノートブックの質によるのでしょうか？
> 
> > ## Eisuke Mizutaniトピック作成者
> > 
> > 現在のトップスコアは0.9未満です。スコアを改善するためのアイデアが残っていると思います。
> > 
> > > ## floriandev
> > > 
> > > Eisukeさん、素晴らしいノートブックをありがとう！あなたのノートブックを使うことで0.9未満になる可能性はありますか？
> > > 

---

> ## Sparsh Tewatia
> 
> 終了までにどれくらいの時間がかかりますか？時間が許せば、LLAMA3とGemma 2の2つのLLMのアンサンブルを行うことができます。
> 
> > ## Lorry Zou
> > 
> > ノートブックでは約4時間と記載されていますので、llama3とgemma2のアンサンブルは実行可能のようです。 
> > 
> > > ## Eisuke Mizutaniトピック作成者
> > > 
> > > 私はllama3とgemma2のアンサンブルを9時間以内で実行できました。max_length=2048およびper_device_batch_size=4を使用しました。
> > > 

---

> ## Lorry Zou
> 
> 私が行った全ての作業が無駄になりました…悲しい😅 でも素晴らしい作品です。
> 
> 

---

> ## Sam
> 
> 私はこのノートブックで提供されているのと同じ推論パラメータ（batch_size、max_length）を使ってGemmaモデルを試すことに決めました。（llamaではなくBert-likeモデルを使用している以外はすべて同じです）。このノートブックはT4x2で9時間以上かかっても終わらず、途中で止まってしまいます。
> 
> 25,000の例をGemmaモデルで処理すると、約17時間かかることがわかりました。
> 
> 何が問題かアドバイスをいただけますか？Gemmaモデルの推論を速くするにはどうすればいいですか？
> 
> > ## Eisuke Mizutaniトピック作成者
> > 
> > [@andreysemenovmax](https://www.kaggle.com/andreysemenovmax) 
> > 
> > 8ビット重量とfp16アクティベーションでのGemma2 9bの動的量子化を行うと、提出のために4.5時間以内に実行されます。
> > 
> > 

---

> ## capyun007
> 
> Kaggleの初心者で質問があります。次のコードを実行してDataFrameをCSVファイルとして保存した後：
> 
> submission_df.to_csv('submission.csv', index=False)
> 
> submission.csvファイルはどこにありますか？
> 
> > ## Eisuke Mizutaniトピック作成者
> > 
> > ノートブックをコミット&保存すると、出力タブに表示されるはずです。インタラクティブセッションで実行した場合は、作業ディレクトリ（./submission.csv）で確認できます。
> > 
> > > ## capyun007
> > > 
> > > 自分のノートブックをコミット&保存しましたが、出力タブに表示されません。
> > > 

---

> ## kanishka sriramoju
> 
> こんにちは、私はここで初心者です。あなたがKaggle入力ディレクトリから事前にトレーニングされたモデルを読み込んだことを見ました。この競技では、ノートブックが独立した環境で実行され、これらの作成されたディレクトリにアクセスできないということはありますか？
> 
> > ## Eisuke Mizutaniトピック作成者
> > 
> > それらのデータセットは公開にしているので、提出時にはアクセスできるはずです。
> > 
> > 